In [1]:
import pandas as pd
import datetime
import logging
from sklearn.model_selection import KFold
import argparse
import json
import numpy as np

import sys
sys.path.append('..')
from utils import load_datasets, load_target
from logs.logger import log_best
from models.lgbm import train_and_predict
import os
os.chdir('../')

%matplotlib inline

In [2]:
os.getcwd()

'C:\\Users\\takuy\\work\\work_tokumoto\\splatoon_competition'

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', default='./configs/default_v01.json')
options = parser.parse_args(args=[])
config = json.load(open(options.config))

feats = config['features']
logging.debug(feats)

target_name = config['target_name']

In [8]:
X_train_all, X_test = load_datasets(feats, target_name)
y_train_all = load_target(target_name)

In [9]:
X_train_all.head()

,A1-level,A2-level,A3-level,A4-level,B1-level,B2-level,B3-level,B4-level,enc_period,enc_game-ver,...,B_rank_max,B_rank_min,A_level_mean,A_level_std,A_level_max,A_level_min,B_level_mean,B_level_std,B_level_max,B_level_min
0,139,118.0,13.0,10.0,28,26.0,68.0,31.0,0.515152,0.525161,...,0.0,0.0,70.00,68.102863,139.0,10.0,38.25,19.939492,68.0,26.0
1,198,77.0,198.0,123.0,83,118.0,168.0,151.0,0.466667,0.524499,...,0.0,0.0,149.00,59.615434,198.0,77.0,130.00,37.585458,168.0,83.0
2,114,68.0,225.0,107.0,50,163.0,160.0,126.0,0.600000,0.525747,...,9.0,7.0,128.50,67.441333,225.0,68.0,124.75,52.582475,163.0,50.0
3,336,131.0,189.0,41.0,273,189.0,194.0,391.0,0.571429,0.523781,...,0.0,0.0,174.25,123.836922,336.0,41.0,261.75,94.365866,391.0,189.0
4,299,97.0,96.0,136.0,101,45.0,246.0,160.0,0.580000,0.524329,...,12.0,12.0,157.00,96.481432,299.0,96.0,138.00,85.957354,246.0,45.0


In [10]:
# 訓練データとテストデータの列を確認
print(X_train_all.columns)
print(X_test.columns)

Index(['A1-level', 'A2-level', 'A3-level', 'A4-level', 'B1-level', 'B2-level',
       'B3-level', 'B4-level', 'enc_period', 'enc_game-ver', 'enc_lobby-mode',
       'enc_lobby', 'enc_mode', 'enc_stage', 'enc_A1-weapon', 'enc_A1-rank',
       'enc_A2-weapon', 'enc_A2-rank', 'enc_A3-weapon', 'enc_A3-rank',
       'enc_A4-weapon', 'enc_A4-rank', 'enc_B1-weapon', 'enc_B1-rank',
       'enc_B2-weapon', 'enc_B2-rank', 'enc_B3-weapon', 'enc_B3-rank',
       'enc_B4-weapon', 'enc_B4-rank', 'A_charger_count', 'B_charger_count',
       'A_rank_mean', 'A_rank_std', 'A_rank_max', 'A_rank_min', 'B_rank_mean',
       'B_rank_std', 'B_rank_max', 'B_rank_min', 'A_level_mean', 'A_level_std',
       'A_level_max', 'A_level_min', 'B_level_mean', 'B_level_std',
       'B_level_max', 'B_level_min'],
      dtype='object')
Index(['A1-level', 'A2-level', 'A3-level', 'A4-level', 'B1-level', 'B2-level',
       'B3-level', 'B4-level', 'enc_period', 'enc_game-ver', 'enc_lobby-mode',
       'enc_lobby', 'enc_mode'

In [11]:
# 訓練データとテストデータの列を確認
print(X_train_all.shape)
print(X_test.shape)

(66125, 48)
(28340, 48)


In [12]:
#　訓練データに欠損がないことの確認
X_train_all.isnull().sum().sum()

0

In [9]:
#　テストデータに欠損がないことの確認
X_test.isnull().sum().sum()

0

In [10]:
print("-------------------train-------------------")
print(X_train_all.isnull().sum())
print("-------------------test-------------------")
print(X_test.isnull().sum())

-------------------train-------------------
A1-level          0
A2-level          0
A3-level          0
A4-level          0
B1-level          0
B2-level          0
B3-level          0
B4-level          0
enc_period        0
enc_game-ver      0
enc_lobby-mode    0
enc_lobby         0
enc_mode          0
enc_stage         0
enc_A1-weapon     0
enc_A1-rank       0
enc_A2-weapon     0
enc_A2-rank       0
enc_A3-weapon     0
enc_A3-rank       0
enc_A4-weapon     0
enc_A4-rank       0
enc_B1-weapon     0
enc_B1-rank       0
enc_B2-weapon     0
enc_B2-rank       0
enc_B3-weapon     0
enc_B3-rank       0
enc_B4-weapon     0
enc_B4-rank       0
dtype: int64
-------------------test-------------------
A1-level          0
A2-level          0
A3-level          0
A4-level          0
B1-level          0
B2-level          0
B3-level          0
B4-level          0
enc_period        0
enc_game-ver      0
enc_lobby-mode    0
enc_lobby         0
enc_mode          0
enc_stage         0
enc_A1-weapon     0


In [14]:
display(X_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28340 entries, 0 to 28339
Data columns (total 30 columns):
A1-level          28340 non-null int64
A2-level          28340 non-null float64
A3-level          28340 non-null float64
A4-level          28340 non-null float64
B1-level          28340 non-null int64
B2-level          28340 non-null float64
B3-level          28340 non-null float64
B4-level          28340 non-null float64
enc_period        28340 non-null float64
enc_game-ver      28340 non-null float64
enc_lobby-mode    28340 non-null float64
enc_lobby         28340 non-null float64
enc_mode          28340 non-null float64
enc_stage         28340 non-null float64
enc_A1-weapon     28340 non-null float64
enc_A1-rank       28340 non-null float64
enc_A2-weapon     28340 non-null float64
enc_A2-rank       28340 non-null float64
enc_A3-weapon     28340 non-null float64
enc_A3-rank       28340 non-null float64
enc_A4-weapon     28340 non-null float64
enc_A4-rank       28340 non-null f

None